# Timer

The `timer` module provides `Chronometer` for measuring time intervals and class `Timer` for accessing 
hardware timers.

## Chronometer

Create, stop, resume, reset ...

In [1]:
from timer import Chronometer
from time import sleep

chrono = Chronometer()

for i in range(5):
    print(chrono.elapsed_time)
    sleep(0.01)
    
chrono.stop()
print("stopped, elapsed time so far", chrono.elapsed_time)
chrono.resume()
sleep(0.5)
print("resumed", chrono.elapsed_time)
chrono.reset()
print("after reset", chrono.elapsed_time)
sleep(0.5)
print("running", chrono.elapsed_time)

0.0
0.01
0.02
0.03
0.04
stopped, elapsed time so far 0.05
resumed 0.55
after reset 0.0
running 0.5


With context manager:

In [2]:
from timer import Chronometer
from time import sleep

with Chronometer() as c:
    sleep(0.7)
    print(c.elapsed_time)

0.7


Deinitializing works as usual. However, since Chronometers are realized entirely in software, the garbage collector takes care of life cycle management. Internally the Chronometer uses a single 64-bit value to store its state.

In [3]:
from timer import Chronometer

c = Chronometer()
c.deinit()
# raises ValueError
c.elapsed_time

Traceback (most recent call last):
  File "<stdin>", line 6, in <module>
ValueError: Object has been deinitialized and can no longer be used. Create a new object.


## Timer

Timers are provided by the hardware. The nRF52840 has five hardware timers of which one is reserved for the "SoftDevice" (Bluetooth). This leaves up to four for the application.

API mimicks `threading.Timer` API.

**BUG:** Exceptions in interrupt callback are silently ignored.

In [4]:
%softreset

from timer import Timer, Chronometer
from time import sleep
import micropython

# required for error handling in callback functions
micropython.alloc_emergency_exception_buf(100)

c = Chronometer()
n = 0

def call_back(timer):
    # Note: since the hardware has already reset the timer when invoking the call_back,
    #       timer.elapsed_time is (nearly) zero
    global n
    n += 1
    print("Callback, n={}, elapsed={}s".format(n, c.elapsed_time))
    if n >= 3:
        timer.cancel()
        print("cancelled")
    
    
t = Timer(interval=0.7, function=call_back, mode=Timer.PERIODIC)
t.start()
print("started")
sleep(0.5)
print("after 500 ms", t.elapsed_time, "sec elapsed")

# wait for callback (otherwise Jupyter thinks the program has ended and does not wait for the output)
for i in range(40):
    # sleep in small increments to give the Python VM a chance to run the callback
    sleep(0.1)

started
after 500 ms 0.499749 sec elapsed
Callback, n=1, elapsed=0.8s
Callback, n=2, elapsed=1.401s
Callback, n=3, elapsed=2.102s
cancelled


### Danger Zone: Fast Interrupts

The default behavior is to schedule callbacks until the MicroPython VM is ready. Specifying `fast=True` executes the callback immediately. Although this avoids a delay (and is critical in some situation), fast interrupts have to be used with **great care**. In particular, no memory allocation (heap) is permitted in "fast" callbacks. I.e. **no objects may be created**. Also some libraries  allocate memory (e.g. `print`!) and hence cannot be used from fast callbacks.

Use fast callbacks at your own risk and only if you fully understand the implications. Consult the [Micropython Documentation](https://docs.micropython.org/en/latest/reference/isr_rules.html) for more information.

In [5]:
%softreset

from timer import Timer, Chronometer
from time import sleep
import micropython 

# required for error handling in callbacks
micropython.alloc_emergency_exception_buf(100)

c = Chronometer()
n = 0

def call_back_2(arg):
    # called by the scheduler, ok to allocate memory and use print
    global c
    print("Callback 2, n={}, elapsed={} sec".format(n, c.elapsed_time))

def call_back_1(timer):
    # Do the urgent stuff here (control the turbo booster, whatever your app demands)
    # nothing urgent - just for illustration
    global n
    n += 1
    if n >= 3:
        timer.cancel()
    # schedule stuff that may allocate memory for when the MicroPython VM is ready
    micropython.schedule(call_back_2, 0)
        
t = Timer(interval=1.2, function=call_back_1, mode=Timer.PERIODIC, fast=True)
t.start()
print("started")
sleep(0.3)
print("after 0.3 sec:", t.elapsed_time, "sec elapsed")

# wait for callback (otherwise Jupyter thinks the program has ended and does not wait for the output)
for i in range(40):
    # sleep in small increments to give the Python VM a chance to run the callback
    sleep(0.1)

started
after 0.3 sec: 0.299266 sec elapsed
Callback 2, n=1, elapsed=1.2 sec
Callback 2, n=2, elapsed=2.399 sec
Callback 2, n=3, elapsed=3.598 sec


### Check if code allocates memory

Use the code below to check if code allocates memory as a diagnostic if it's save to use in a fast interrupt handler.

In [6]:
%softreset

import micropython

x = None

micropython.heap_lock()

# safe, since x has been created before
x = 3+4

# print is safe, apparently
print("hello", 1, x, 3.4)

micropython.heap_unlock()

print("test complete")

hello 1 7 3.4
test complete


In [7]:
%softreset

import micropython

micropython.heap_lock()

# raises MemoryError since a new object (x) is created
# x = 3

# format raises MemoryError
print("int={} float={}".format(3, 5.2))

micropython.heap_unlock()

print("test complete")

MemoryError: 


### Check how many timers are available

In [8]:
%softreset

from timer import Timer

for i in range(5):
    try:
        timer = Timer()
        print("created {} Timer(s)".format(i+1))
    except ValueError as e:
        print("attempt to create timer resulted in an error:", e)

created 1 Timer(s)
created 2 Timer(s)
created 3 Timer(s)


Traceback (most recent call last):
  File "<stdin>", line 9, in <module>
  File "<stdin>", line 6, in <module>
RuntimeError: All timers are in use
